# Interacting with Recommenders and Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Event Tracking - Keeping up with Evolving User Intent](#event-tracking)
1. [Real-time Events](#real-time)
1. [Category Filter](#filter)
1. [Batch Recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have 2 Recommenders and one deployed campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 


In this Notebook we will interact with Recommenders and Campaigns and get recommendatiosn. We will interact with filters and send live data to Amazon Personalize to see the effect on recommendations.

![Workflow](images/image3.png)

To run this notebook, you need to have run the previous notebooks, `01_Data_Layer.ipynb`, and `02_Training_Layer.ipynb`, where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions and campaigns. At the end of that notebook, you saved some of the variable values, which you now need to load into this notebook.

As you work with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [ ]:
#retrieves previously stored variables 
%store -r 

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Adding some helper functions to make results more readable

In [ ]:
def get_item_name_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['name'].values[0]
    return item_name

In [ ]:
def get_item_category_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['category'].values[0]
    return item_name

In [ ]:
def get_item_style_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['style'].values[0]
    return item_name

In [ ]:
# def add_item_name_to_df ( df ):
#     test = df.copy()

#     test['ITEM_NAME'] = test.apply(
#           lambda row:
#             get_item_name_from_id(row['ITEM_ID'] ), axis=1
#       )
    
#     display(test)
#     return (test)

## Interact with Recommenders <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the recommenders have been trained, lets have a look at the recommendations we can get for our users!

### "Customers who viewed X also viewed" Recommender

"Customers who viewed X also viewed" requires an item and a user as input, and it will return customers also viewed based on an item that you specify.

The cells below will handle getting recommendations from the "Customers who viewed X also viewed" Recommender and rendering the results. Let's see what the recommendations are for an item.

We will be using the `recommenderArn`, the `itemId`, the `userId`, as well as the number or results we want, `numResults`.

#### Select a User

We'll just pick a random user for simplicity (there are about 6,000 users with user IDs assigned in sequential order). Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

In [ ]:
user_id = 555
user_metadata_df[user_metadata_df['id']==user_id]

We'll just pick a random product for simplicity. Feel free to change the `product_id` below and execute the following cells with a different product to get a sense for how the recommendations change.

In [ ]:
product_id = item_metadata_df.sample(1)["id"].values[0]

print ('productId: ', product_id)

get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = viewed_x_also_viewed_arn,
    itemId = str(product_id),
    userId = str(user_id),
    numResults = 20
)


In [ ]:
item_list = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for similar movies render in a nice dataframe. Again, let's create a helper function to achieve this.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_viewed_x(recommendations_df, item_id, user_id):
    # Get the item name
    original_item_name = get_item_name_from_id(item_id)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = viewed_x_also_viewed_arn,
        itemId = str(item_id),
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [original_item_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

Now, let's test the helper function with several different items. Let's sample some data from our dataset to test our "Customers who viewed X also viewed" Recommender. Grab 5 random items from our dataframe.

In [ ]:
samples = item_metadata_df.sample(5)
samples

In [ ]:
viewed_x_also_viewed_recommendations_df = pd.DataFrame()
items = samples.id.tolist()

for item in items:
    viewed_x_also_viewed_recommendations_df = get_new_recommendations_df_viewed_x(viewed_x_also_viewed_recommendations_df, item, user_id)

viewed_x_also_viewed_recommendations_df

Based on the random product selected above, do the similar item recommendations from Personalize make sense? Keep in mind that the "Customers who viewed X also viewed" recommendations are based on the interactions we generated as input into the solution creation process above.

You may notice that some of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with a small dataset).

### "Recommended for you" Recommender

"Recommended for you" supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

With this use case, Amazon Personalize automatically filters items the user purchased based on the `userId` that you specify and "Purchase" events. For better performance, include "Purchase" events along with the required "View" events. 

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20
)

In [ ]:
item_list = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)


Since "Recommended for you" relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [ ]:
samples = user_metadata_df.sample(3)
samples

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

"Recommended for you" requires only a user as input, and it will return items that are relevant for that particular user. In this particular case the item is a product.

The cells below will handle getting recommendations from the "Recommended for you" Recommender and rendering the results. 

We will be using the `recommenderArn`, the `userId` as well as the number or results we want, `numResults`.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_recommended_for_you(recommendations_df, user_id):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommended_for_you_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        item_name = get_item_name_from_id(item['itemId'])
        recommendation_list.append(item_name)
    new_rec_df = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_df], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()
users = samples.id.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_recommended_for_you(recommendations_df_users, user)

recommendations_df_users

## Applying the discount context

We'll get the user recommendations when discount context is applied for comparison. This is a using the "contextual metadata" feature of Amazon Personalize. In this case our context is whether a discount was applied.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommended_for_you_arn,
    userId = str(user_id),
    numResults = 20,
    context = {'DISCOUNT':'Yes'} # Here we provide the context for the recommendations
)

item_list_context = get_recommendations_response['itemList']

df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list_context ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list_context ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

Let us compare it to the previous set of recommendations without context.

In [ ]:
df = pd.DataFrame()
df['Item (No context)'] = [ itm['itemId'] for itm in item_list ]
df['Name (No context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]

df['Item (Context)'] = [ itm['itemId'] for itm in item_list_context ]
df['Name (Context)'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list_context ]
display (df)

## Interact with Campaigns <a class="anchor" id="interact-campaigns"></a>
[Back to top](#top)

Now that the reranking campaign is deployed and active, we can start to get recommendations via an API call. 

### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For an ECOMMERCE application you want dynamically render a personalized shelf/rail/carousel based on some information (category, style, season, etc...). This may not be information that you have in your metadata, so a item metadata filter will not work, however you may have this information within you system to generate the item list. You can use this campaign to rerank the products listed for each category and the featured products list as well as reranking catalog search results displayed in a search widget.

To demonstrate this, we will use the same user from before and a random collection of items.

#### Get Featured Products List

First let's get the list of featured products from the Products data.

In [ ]:
featured_products = item_metadata_df[item_metadata_df['featured']==True]

#### ReRank Featured Products

Using the featured products list just retrieved, first we'll create a list of item IDs that we want to rerank for a specific user. This reranking will allow us to provide ranked products based on the user's behavior. These behaviors should be consistent the same persona that was mentioned above (since we're going to use the same `user_id`).

In [ ]:
unranked_product_ids = list (featured_products['id'])

df = pd.DataFrame()
df['Item'] = [ itm for itm in unranked_product_ids ]
df['Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]
df['Category'] = [ get_item_category_from_id ( itm) for itm in unranked_product_ids ]
df['Style'] = [ get_item_style_from_id ( itm) for itm in unranked_product_ids ]
display (df)

Now let's have Personalize rank the featured product IDs based on our random user.

In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn=rerank_campaign_arn,
    inputList=unranked_product_ids,
    userId=str(user_id)
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

Notice that in this response we have a score field returned with each itemId. For all recipes except SIMS and Popularity-Count, Personalize [calculates a score](https://docs.aws.amazon.com/personalize/latest/dg/getting-real-time-recommendations.html) for each recommended item. Score values are between 0.0 and 1.0. 

Let us compare it to the original list:

In [ ]:
df = pd.DataFrame()
df['Original List'] = [ itm for itm in unranked_product_ids]
df['Original List Name'] = [ get_item_name_from_id ( itm) for itm in unranked_product_ids ]

df['Reranking'] = [ itm['itemId'] for itm in reranked]
df['Reranking Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in reranked ]

display (df)

Are the reranked results different than the original results from the Search service? Notice that we are also given a score for each item but this time the score values are larger. This is because scores for personalized-ranking results are calculated just across the items being reranked. Experiment with a different `user_id` in the cells above to see how the item ranking changes.

## Event Tracking - Keeping up with Evolving User Intent<a class="anchor" id="event-tracking"></a>
[Back to top](#top)

Up to this point we have trained and deployed three Amazon Personalize campaigns based on historical data that we generated in this workshop. This allows us to make related product, user recommendations, and rerank product lists based on already observed behavior of our users. However, user intent often changes in real-time such that what products the user is interested in now may be different than what they were interested in a week ago, a day ago, or even a few minutes ago. Making recommendations that keep up with evolving user intent is one of the more difficult challenges with personalization. Fortunately, Amazon Personalize has a mechanism for this exact issue.

Amazon Personalize supports the ability to send real-time user events (i.e. clickstream) data into the service.
Personalize uses this event data to improve recommendations. It will also save these events and automatically
include them when solutions for the same dataset group are re-created (i.e. model retraining).

For our sample interactions dataset we have common e-commerce events such as 'View' (product), 'AddToCart', 'Purchase', and others. These can be streamed in real-time from client applications as they occur to a Personalize Event Tracker.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [ ]:
event_tracker_response = personalize.create_event_tracker(
    datasetGroupArn=dataset_group_arn,
    name='amazon-personalize-immersionday-event-tracker'
)

event_tracker_arn = event_tracker_response['eventTrackerArn']
event_tracking_id = event_tracker_response['trackingId']

print('Event Tracker ARN: ' + event_tracker_arn)
print('Event Tracking ID: ' + event_tracking_id)

### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker. Since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. This challenge can be solved by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows applications to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, client applications can pass the provisional user ID to the user management system when creating a new user account. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

### Test Purchased Products Filter

To test our purchased products filter, we will request ranked recommendations for a random user. Then we will send a `Purchase` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter. The purchased product would be excluded from the new recommendations.

In [ ]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = 456
display(user_metadata_df[user_metadata_df['id']==user_id])

In [ ]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn=rerank_campaign_arn,
    inputList=unranked_product_ids,
    userId = str(user_id)
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [ ]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')
print(f'Product name: {get_item_name_from_id ( product_id_to_purchase)}')

Next let's send a `Purchase` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [ ]:
response = personalize_events.put_events(
    trackingId = event_tracking_id,
    userId = str(user_id),
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'Purchase',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for Purchase event to become consistent.
time.sleep(10)

print(json.dumps(response, indent=2))

Finally, let's retrieve recommendations for the user again but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.

In [ ]:
get_recommendations_response = personalize_runtime.get_personalized_ranking(
    campaignArn=rerank_campaign_arn,
    inputList=unranked_product_ids,
    userId = str(user_id),
    filterArn = filter_purchase_arn
)

item_list = get_recommendations_response['personalizedRanking']
df = pd.DataFrame()
df['Item'] = [ itm['itemId'] for itm in item_list ]
df['Name'] = [ get_item_name_from_id ( itm['itemId']) for itm in item_list ]
df['Category'] = [ get_item_category_from_id ( itm['itemId']) for itm in item_list ]
df['Style'] = [ get_item_style_from_id ( itm['itemId']) for itm in item_list ]
display (df)

The following code will raise an assertion error if the product we just purchased is still recommended.

In [ ]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

## Category Filter<a class="anchor" id="filter">
[Back to top](#top)

To test our purchased products filter, we will request recommendations for a random user with a particular category.

In [ ]:
def get_new_recommendations_df_by_dynamic_filter(recommendations_df, user_id, category_filter_arn, filter_values):
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = recommended_for_you_arn,
        userId = str(user_id),
        filterArn = filter_category_arn,
        filterValues = { "CATEGORY": "\"" + filter_values + "\""}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_item_name_from_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = filter_category_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

We will select the first 7 categories for this example.

In [ ]:
unique_category_field_values = list(item_metadata_df['category'].unique()[:7])
unique_category_field_values

In [ ]:
# Iterate through Genres
recommendations_df_category = pd.DataFrame()
for category in unique_category_field_values:
    recommendations_df_category = get_new_recommendations_df_by_dynamic_filter(recommendations_df_category, user_id, filter_category_arn , category)
    
recommendations_df_category

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the Personalized Ranking solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/recommendations-batch.html). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": <S3 input path>}},
    jobOutput = 
       {"s3DataDestination": {"path": <S3 output path>}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for Personalized Ranking, it looks like:


```JSON
{"userId": "891", "itemList": ["27", "886", "101"]}
{"userId": "445", "itemList": ["527", "55", "901"]}
{"userId": "71", "itemList": ["27", "351", "101"]}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"891","itemList":["27","886","101"]},"output":{"recommendedItems":["27","101","886"],"scores":[0.48421,0.28133,0.23446]}}
{"input":{"userId":"445","itemList":["527","55","901"]},"output":{"recommendedItems":["901","527","55"],"scores":[0.46972,0.31011,0.22017]}}
{"input":{"userId":"71","itemList":["29","351","199"]},"output":{"recommendedItems":["351","29","199"],"scores":[0.68937,0.24829,0.06232]}}

```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [ ]:
# We will use the same users from before
print (users)
# Write the file to disk
json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '", "itemList":'+json.dumps(unranked_product_ids)+'}\n')

In [ ]:
# Showcase the input file:
!cat $json_input_filename

Upload the file to S3 and save the path as a variable for later.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = rerank_solution_version_arn,
    jobName = "POC-Batch-Inference-Job-PersonalizedRanking_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [ ]:
%%time

current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_item_name_from_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You can choose to head to `04_Operations_Layer.ipynb` to go deeper into ML Ops and what a production solution can look like with an automation pipeline.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `05_Clean_Up_Resources.ipynb`.

In [ ]:
%store event_tracker_arn